In [ ]:
"""
This module implement util classes for reading and visalizing the DREAM dataset using Python 3. 
"""
import os
import json
import io
import numpy as np
import pandas as pd
from math import sin,cos
from functools import reduce
import cv2
from matplotlib import pyplot as plt

def open(path):
    with io.open(path) as f:
        return Intervention(json.load(f))

def withlast(iterable):
	i = iter(iterable)
	try:
		v = next(i)
		for vv in i: 
			yield v+(False,) if isinstance(v,tuple) else (v,False)
			v = vv
		yield v+(True,) if isinstance(v,tuple) else (v,True)
	except StopIteration:
		pass

class Intervention(dict):

    def __init__(self, data):
        for key in data:
            self[key] = self.__fixNan(data[key])

    def __fixNan(self,data):
        if data is None:
            return float('nan')
        elif isinstance(data,dict):
            for key in data:
                data[key] = self.__fixNan(data[key])
        elif isinstance(data,list):
            for i, v in enumerate(data):
                data[i] = self.__fixNan(v)
        return data

    def __repr__(self):
        return 'Intervention recording with {} samples'.format(self.sampleCount())

    def sampleCount(self):
        return len(self['skeleton']['head']['x'])

    def structure(self,dic=None,linewidth=50,tab=1):
        if dic is None: dic = self
        s = ['{']
        for key,val,islast in withlast(dic.items()):
            if isinstance(val,dict):
                s.append('"{0}": {1}{2}'.format(key, self.structure(val,linewidth,tab+1), '' if islast else ','))
            elif isinstance(val,list):
                s.append('"{0}": {1}{2}'.format(key, '[]', '' if islast else ','))
            elif isinstance(val,str):
            	s.append('"{0}": "{1}"{2}'.format(key, val, '' if islast else ','))
            elif val is None:
                s.append('"{0}": null{2}'.format(key, '' if islast else ','))
            else:
                s.append('"{0}": {1}{2}'.format(key, val, '' if islast else ','))
        if len(s)==1:
            return "{}"
        elif reduce(lambda n,s: n+len(s),s,0) < linewidth:
            return ''.join(s) + '}'
        else:
            return ('\n'+(' '*4*tab)).join(s) + '\n' + (' '*4*(tab-1)) + '}'

    def gaze(self,distance=1):
        headPose = [np.array([v]).transpose() for v in zip(self['skeleton']['head']['x'],self['skeleton']['head']['y'],self['skeleton']['head']['z'])]
        xrot = [self.xrot(gaze) for gaze in zip(self['head_gaze']['rx'],self['head_gaze']['ry'],self['head_gaze']['rz'])]
        return [np.matmul(rot,pose) for rot,pose in zip(xrot,headPose)]
        
    def xrot(self,gaze):
        v = gaze[0]
        return np.array([[0,0,1],[-sin(v),cos(v),0],[cos(v),sin(v),0]])
    
    def to_csv(self,*args,**kwargs):
        return self.to_dataFrame().to_csv(*args,**kwargs)
    
    def to_dataFrame(self):
        return pd.DataFrame.from_dict(dict(self.columns()))
    
    def columns(self,d=None,parent_name='',trim=True):
        if d is None: d=self
        if trim == True: d.trim()
        for k, v in d.items():
            if '$' in k: continue
            if isinstance(v,dict):
                for c, cc in self.columns(v,parent_name + '_' + k if parent_name else k,False):
                    yield c, cc
            else:
                yield parent_name + '_' + k if parent_name else k, v
                
    def trim(self):
        """Guarantees that all time dependent data has the same length, extending arrays where necessary."""
        length = 0
        cols = list(self.columns(trim=False))
        for c, v in cols:
            if isinstance(v,list): length = max(length,len(v))
        for c, v in cols:
            while isinstance(v,list) and len(v) > 1 and len(v) < length:
                v.append(float('nan'))

In [ ]:
# import dataset_tools as dataset

# import pandas as pd
# import numpy as np
# os.listdir(home)

In [ ]:
def preprocess(df):
    drop_list=["ados_preTest_communication", "ados_preTest_interaction", "ados_preTest_module", "ados_preTest_play", "ados_preTest_protocol", "ados_preTest_socialCommunicationQuestionnaire", "ados_preTest_stereotype", "condition", "frame_rate", "participant_gender", "participant_id", "skeleton_elbow_left_confidence", "skeleton_elbow_right_confidence", "skeleton_hand_left_confidence", "skeleton_hand_right_confidence", "skeleton_head_confidence", "skeleton_sholder_center_confidence", "skeleton_sholder_left_confidence", "skeleton_sholder_right_confidence", "skeleton_wrist_left_confidence", "skeleton_wrist_right_confidence", "task_ability", "task_difficultyLevel", "task_end", "task_index", "task_start", "time"]
    df.drop(drop_list,axis=1,inplace=True)
    df.dropna(axis=0,inplace=True)
    df=df[::5].reset_index(drop=True)
    return df

In [ ]:

# data=open("/kaggle/input/dream-dataset-part1/Part 1 Users/User 11/User 11_2_diagnosis abilities_20170315_090533.665000.json")
# df=preprocess(data.to_dataFrame())
# # print(df.head())
# # fig, ax = plt.subplots(1,1,figsize=(12, 7))
# skeleton = data['skeleton']
# frame = 25*16
# for key in skeleton:
#     print(skeleton[key]['x'][frame],skeleton[key]['y'][frame])
#     x,y = skeleton[key]['x'][frame],skeleton[key]['y'][frame]
#     ax.plot(x,y,'o')
#     ax.text(x,y,key)
# ax.axis([-300,500,-700,0])

In [ ]:
# # import matplotlib.pyplot as plt

# # Coordinates
# coordinates = [
#     (-146.001, -564.381281),
#     (56.9436, -579.478134),
#     (-6.61139, -674.218252),
#     (262.33, -650.46385),
#     (-19.8384, -272.785714),
#     (-38.4072, -420.667455)
# ]

# # Image size
# image_size = 256

# # Calculate the range of the coordinates
# x_min = min(coord[0] for coord in coordinates)
# x_max = max(coord[0] for coord in coordinates)
# y_min = min(coord[1] for coord in coordinates)
# y_max = max(coord[1] for coord in coordinates)

# # Calculate the scale factors for the transformation
# x_scale = image_size / (x_max - x_min)
# y_scale = image_size / (y_max - y_min)

# # Create a new figure and axis
# fig, ax = plt.subplots()

# # Set the limits of the plot
# ax.set_xlim(0, image_size)
# ax.set_ylim(0, image_size)

# # Plot each coordinate as a dot after applying the transformation
# for coord in coordinates:
#     x, y = coord
#     # Apply the transformation
#     x_pixel = (x - x_min) * x_scale
#     y_pixel = (y - y_min) * y_scale
#     ax.plot(x_pixel, y_pixel, 'bo')  # 'bo' represents blue dots

# # Display the plot
# plt.show()


In [ ]:
# import cv2
# import numpy as np

# # Define the image size and resolution
# image_width = 256
# image_height = 256
# image_channels = 3  # Assuming an RGB image, change if necessary

# # Create an empty image canvas
# image = np.zeros((image_height, image_width, image_channels), dtype=np.uint8)

# # Define skeleton colors
# skeleton_color = (0, 255, 0)  # Green color

# # Define gaze color
# gaze_color = (0, 0, 255)  # Red color

# # Example skeleton data (assuming 2D coordinates)
# skeleton_data = {
#     'head': (50, 50),
#     'shoulder': (50, 100),
#     'hand_left': (30, 120),
#     'hand_right': (70, 120),
#     'leg_left': (40, 200),
#     'leg_right': (60, 200)
# }

# # Example gaze data (assuming gaze direction coordinates)
# gaze_data = {
#     'eye_left': (45, 55),
#     'eye_right': (55, 55),
#     'gaze_direction': (50, 40)
# }

# # Draw skeleton on the image canvas
# for joint, (x, y) in skeleton_data.items():
#     cv2.circle(image, (x, y), 3, skeleton_color, -1)
#     cv2.putText(image, joint, (x + 5, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, skeleton_color, 2)

# # Draw gaze markers on the image canvas
# for eye in ['eye_left', 'eye_right']:
#     eye_x, eye_y = gaze_data[eye]
#     cv2.circle(image, (eye_x, eye_y), 2, gaze_color, -1)

# gaze_direction_x, gaze_direction_y = gaze_data['gaze_direction']
# cv2.arrowedLine(image, (gaze_direction_x, gaze_direction_y), (eye_x, eye_y), gaze_color, 2)

# # Display the image
# cv2.imshow('Image Representation', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
# image = np.zeros((image_height, image_width), dtype=np.uint8)
# cv2.circle(image, (50, 50), 1, 255, -1)
# plt.imshow(image)
# cv2.imwrite("output.png",image)

In [ ]:
# image_size=36
# def bound(df):
#     for c in list(df.columns)[1:]:
#         mi=np.min(df[c])
#         ma=np.max(df[c])
#         scale = image_size / (ma - mi)
#         df[c]=np.int32((df[c]-mi)*scale)
#     return df

# def getImage(df):
#     ans=[]
#     for index, d in df.iterrows():
#         image = np.zeros((image_height, image_width), dtype=np.uint8)
#         cv2.circle(image, (d['skeleton_elbow_left_x'], d['skeleton_elbow_left_y']), 1, 255, -1)
#         cv2.circle(image, (d['skeleton_elbow_right_x'], d['skeleton_elbow_right_y']), 1, 255, -1)
#         cv2.circle(image, (d['skeleton_hand_left_x'], d['skeleton_hand_left_y']), 1, 255, -1)
#         cv2.circle(image, (d['skeleton_hand_right_x'], d['skeleton_hand_right_y']), 1, 255, -1)
#         cv2.circle(image, (d['skeleton_head_x'], d['skeleton_head_y']), 1, 255, -1)
#         cv2.circle(image, (d['skeleton_sholder_center_x'], d['skeleton_sholder_center_y']), 1, 255, -1)
#         cv2.circle(image, (d['skeleton_sholder_left_x'], d['skeleton_sholder_left_y']), 1, 255, -1)
#         cv2.circle(image, (d['skeleton_sholder_right_x'], d['skeleton_sholder_right_y']), 1, 255, -1)
#         cv2.circle(image, (d['skeleton_wrist_left_x'], d['skeleton_wrist_left_y']), 1, 255, -1)
#         cv2.circle(image, (d['skeleton_wrist_right_x'], d['skeleton_wrist_right_y']), 1, 255, -1)
        
#         cv2.line(image,(d['skeleton_head_x'], d['skeleton_head_y']),(d['skeleton_sholder_center_x'], d['skeleton_sholder_center_y']),255)
#         cv2.line(image,(d['skeleton_sholder_left_x'], d['skeleton_sholder_left_y']),(d['skeleton_sholder_center_x'], d['skeleton_sholder_center_y']),255)
#         cv2.line(image,(d['skeleton_sholder_right_x'], d['skeleton_sholder_right_y']),(d['skeleton_sholder_center_x'], d['skeleton_sholder_center_y']),255)
#         cv2.line(image,(d['skeleton_sholder_left_x'], d['skeleton_sholder_left_y']),(d['skeleton_elbow_left_x'], d['skeleton_elbow_left_y']),255)
#         cv2.line(image,(d['skeleton_hand_left_x'], d['skeleton_hand_left_y']),(d['skeleton_elbow_left_x'], d['skeleton_elbow_left_y']),255)
#         cv2.line(image,(d['skeleton_hand_left_x'], d['skeleton_hand_left_y']),(d['skeleton_wrist_left_x'], d['skeleton_wrist_left_y']),255)
        
#         cv2.line(image,(d['skeleton_sholder_right_x'], d['skeleton_sholder_right_y']),(d['skeleton_elbow_right_x'], d['skeleton_elbow_right_y']),255)
#         cv2.line(image,(d['skeleton_hand_right_x'], d['skeleton_hand_right_y']),(d['skeleton_elbow_right_x'], d['skeleton_elbow_right_y']),255)
#         cv2.line(image,(d['skeleton_hand_right_x'], d['skeleton_hand_right_y']),(d['skeleton_wrist_right_x'], d['skeleton_wrist_right_y']),255)
        
#         ans.append(image)
#     return ans


# df=bound(df.head())
# # video=getImage(df.head())

In [ ]:
image_size=30
# def bound(df):
#     for c in list(df.columns)[1:]:
#         mi=np.min(df[c])
#         ma=np.max(df[c])
#         scale = image_size / (ma - mi)
#         df[c]=(df[c]-mi)*scale
#     return df

def getImage(df):
    ans=[]
    for index, d in df.iterrows():
        a=['skeleton_elbow_left_x','skeleton_elbow_left_y','skeleton_elbow_right_x','skeleton_elbow_right_y','skeleton_hand_left_x','skeleton_hand_left_y','skeleton_hand_right_x','skeleton_hand_right_y','skeleton_head_x','skeleton_head_y','skeleton_sholder_center_x','skeleton_sholder_center_y','skeleton_sholder_left_x','skeleton_sholder_left_y','skeleton_sholder_right_x','skeleton_sholder_right_y','skeleton_wrist_left_x','skeleton_wrist_left_y','skeleton_wrist_right_x','skeleton_wrist_right_y']
        mi=np.min(d[a])
        ma=np.max(d[a])
        if(mi==0 and ma==0):
            ans.append(np.zeros((image_size, image_size, 1), dtype=np.uint8))
            continue
        scale = image_size / (ma - mi)
        d[a]=(d[a]-mi)*scale
        image = np.zeros((image_size, image_size, 1), dtype=np.uint8)
        cv2.circle(image, (int(d['skeleton_elbow_left_x']), int(d['skeleton_elbow_left_y'])), 1, 255, -1)
        cv2.circle(image, (int(d['skeleton_elbow_right_x']), int(d['skeleton_elbow_right_y'])), 1, 255, -1)
        cv2.circle(image, (int(d['skeleton_hand_left_x']), int(d['skeleton_hand_left_y'])), 1, 255, -1)
        cv2.circle(image, (int(d['skeleton_hand_right_x']), int(d['skeleton_hand_right_y'])), 1, 255, -1)
        cv2.circle(image, (int(d['skeleton_head_x']), int(d['skeleton_head_y'])), 1, 255, -1)
        cv2.circle(image, (int(d['skeleton_sholder_center_x']), int(d['skeleton_sholder_center_y'])), 1, 255, -1)
        cv2.circle(image, (int(d['skeleton_sholder_left_x']), int(d['skeleton_sholder_left_y'])), 1, 255, -1)
        cv2.circle(image, (int(d['skeleton_sholder_right_x']), int(d['skeleton_sholder_right_y'])), 1, 255, -1)
        cv2.circle(image, (int(d['skeleton_wrist_left_x']), int(d['skeleton_wrist_left_y'])), 1, 255, -1)
        cv2.circle(image, (int(d['skeleton_wrist_right_x']), int(d['skeleton_wrist_right_y'])), 1, 255, -1)
        
        cv2.line(image,(int(d['skeleton_head_x']), int(d['skeleton_head_y'])),(int(d['skeleton_sholder_center_x']), int(d['skeleton_sholder_center_y'])),255)
        cv2.line(image,(int(d['skeleton_sholder_left_x']), int(d['skeleton_sholder_left_y'])),(int(d['skeleton_sholder_center_x']), int(d['skeleton_sholder_center_y'])),255)
        cv2.line(image,(int(d['skeleton_sholder_right_x']), int(d['skeleton_sholder_right_y'])),(int(d['skeleton_sholder_center_x']), int(d['skeleton_sholder_center_y'])),255)
        cv2.line(image,(int(d['skeleton_sholder_left_x']), int(d['skeleton_sholder_left_y'])),(int(d['skeleton_elbow_left_x']), int(d['skeleton_elbow_left_y'])),255)
        cv2.line(image,(int(d['skeleton_hand_left_x']), int(d['skeleton_hand_left_y'])),(int(d['skeleton_elbow_left_x']), int(d['skeleton_elbow_left_y'])),255)
        cv2.line(image,(int(d['skeleton_hand_left_x']), int(d['skeleton_hand_left_y'])),(int(d['skeleton_wrist_left_x']), int(d['skeleton_wrist_left_y'])),255)
        
        cv2.line(image,(int(d['skeleton_sholder_right_x']), int(d['skeleton_sholder_right_y'])),(int(d['skeleton_elbow_right_x']), int(d['skeleton_elbow_right_y'])),255)
        cv2.line(image,(int(d['skeleton_hand_right_x']), int(d['skeleton_hand_right_y'])),(int(d['skeleton_elbow_right_x']), int(d['skeleton_elbow_right_y'])),255)
        cv2.line(image,(int(d['skeleton_hand_right_x']), int(d['skeleton_hand_right_y'])),(int(d['skeleton_wrist_right_x']), int(d['skeleton_wrist_right_y'])),255)
        
        ans.append(image)
    return ans


# df=bound(df.head())
# video=getImage(df.head())

In [ ]:
# list(df.columns)

In [ ]:
# ['skeleton_elbow_left_x',
#  'skeleton_elbow_left_y',
#  'skeleton_elbow_left_z',
#  'skeleton_elbow_right_x',
#  'skeleton_elbow_right_y',
#  'skeleton_elbow_right_z',
#  'skeleton_hand_left_x',
#  'skeleton_hand_left_y',
#  'skeleton_hand_left_z',
#  'skeleton_hand_right_x',
#  'skeleton_hand_right_y',
#  'skeleton_hand_right_z',
#  'skeleton_head_x',
#  'skeleton_head_y',
#  'skeleton_head_z',
#  'skeleton_sholder_center_x',
#  'skeleton_sholder_center_y',
#  'skeleton_sholder_center_z',
#  'skeleton_sholder_left_x',
#  'skeleton_sholder_left_y',
#  'skeleton_sholder_left_z',
#  'skeleton_sholder_right_x',
#  'skeleton_sholder_right_y',
#  'skeleton_sholder_right_z',
#  'skeleton_wrist_left_x',
#  'skeleton_wrist_left_y',
#  'skeleton_wrist_left_z',
#  'skeleton_wrist_right_x',
#  'skeleton_wrist_right_y',
#  'skeleton_wrist_right_z']

In [ ]:
# for index, d in df.iterrows():
#         image = np.zeros((image_height, image_width), dtype=np.uint8)
#         print(type(d['skeleton_elbow_left_x']))
#         print(int(d['skeleton_elbow_left_x']))
#         break

In [ ]:
# video=getImage(df)


In [ ]:
# a=abs((df['skeleton_wrist_right_y'].head())*0)
# type(a)

In [ ]:
# plt.imshow(video[1])

In [ ]:
dirs = os.listdir("/kaggle/input/dream-dataset-part1/Part 1 Users")
print(dirs)

In [ ]:
videos=[]
home="/kaggle/input/dream-dataset-part1/Part 1 Users"

per_user=10
y=[]
for dir in os.listdir(home):
    n=0
    for dirname, _, filenames in os.walk(os.path.join(home, dir)):
        for filename in filenames:
            if(n==per_user):
                break
            data=open(os.path.join(dirname, filename))
#             print(os.path.join(dirname, filename))
            df=data.to_dataFrame()
            df=preprocess(df)
            if(df.shape[0]==0):
                continue
#             print(df.shape)
            y.append(df["ados_preTest_total"][0])
            videos.append(df.drop(["ados_preTest_total"], axis=1))
            n+=1


In [ ]:
# home="/kaggle/input/dream-dataset-part-2/Part 2 Users"
# per_user=3
# for dir in os.listdir(home):
# #     n=0
#     for dirname, _, filenames in os.walk(os.path.join(home, dir)):
#         for filename in filenames:
# #             if(n==per_user):
# #                 break
#             data=open(os.path.join(dirname, filename))
# #             print(os.path.join(dirname, filename))
#             df=data.to_dataFrame()
#             df=preprocess(df)
#             if(df.shape[0]==0):
#                 continue
# #             print(df.shape)
#             videos.append(df)
# #             n+=1


In [ ]:
# home="/kaggle/input/dream-dataset-part-3/Part 3 Users"
# per_user=3
# for dir in os.listdir(home):
# #     n=0
#     for dirname, _, filenames in os.walk(os.path.join(home, dir)):
#         for filename in filenames:
# #             if(n==per_user):
# #                 break
#             data=open(os.path.join(dirname, filename))
# #             print(os.path.join(dirname, filename))
#             df=data.to_dataFrame()
#             df=preprocess(df)
#             if(df.shape[0]==0):
#                 continue
# #             print(df.shape)
#             videos.append(df)
# #             n+=1


In [ ]:
# home="/kaggle/input/dream-dataset-part-4/Part 4 Users"
# per_user=3
# for dir in os.listdir(home):
# #     n=0
#     for dirname, _, filenames in os.walk(os.path.join(home, dir)):
#         for filename in filenames:
# #             if(n==per_user):
# #                 break
#             data=open(os.path.join(dirname, filename))
# #             print(os.path.join(dirname, filename))
#             df=data.to_dataFrame()
#             df=preprocess(df)
#             if(df.shape[0]==0):
#                 continue
# #             print(df.shape)
#             videos.append(df)
# #             n+=1


In [ ]:
print(len(videos))
max_frames=100
padded_dataframes = []
for i in range(len(videos)):
    max_frames = max(max_frames,len(videos[i]))
print(max_frames)

In [ ]:
videos[1].shape

In [ ]:
def getBest(df):
    diff=np.linalg.norm(df.diff(axis=0).drop(0), axis=1)
    ind=diff.argmax()
    return ind

In [ ]:
max_frames=4000
# padded_dataframes = []

for i in range(len(videos)):
    padding_length = max_frames - len(videos[i])
    if(padding_length<=0):
        videos[i] = (videos[i][:max_frames]).to_numpy()
#         ind = getBest(videos[i])
#         if(ind-max_frames//2>-1 and ind+max_frames//2<len(videos[i])):
#             videos[i]=(videos[i][ind-max_frames//2:ind+max_frames//2]).reset_index(drop=True)
#         elif(ind-max_frames//2<0):
#             videos[i]=(videos[i][:max_frames]).reset_index(drop=True)
#         else:
#             videos[i]=(videos[i][-max_frames:]).reset_index(drop=True)
    else:
        videos[i] = (pd.concat([videos[i], pd.DataFrame(np.zeros((padding_length, videos[i].shape[1])), columns=videos[i].columns)]).reset_index(drop=True)).to_numpy()
    
# videos[0]["ados_preTest_total"]

In [ ]:
videos=np.array(videos)
y=np.array(y)
print(videos.shape)
print(y.shape)

In [ ]:
y[3]

In [ ]:
videos_transposed = np.transpose(videos, (1, 0, 2))
X = np.split(videos_transposed, 4000 // 100, axis=0)
X = np.array(X)
X = np.transpose(X, (2, 0, 1, 3))
X = np.array(X)
print(X.shape)

In [ ]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        # conv block
        self.conv2Dblock = nn.Sequential(
            # 1. conv block
            nn.Conv2d(in_channels=1,
                                   out_channels=16,
                                   kernel_size=3,
                                   stride=1,
                                   padding=1
                                  ),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.2),
            # 2. conv block
            nn.Conv2d(in_channels=16,
                                   out_channels=32,
                                   kernel_size=3,
                                   stride=1,
                                   padding=1
                                  ),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.Dropout(p=0.2),
            # 3. conv block
            nn.Conv2d(in_channels=32,
                                   out_channels=64,
                                   kernel_size=3,
                                   stride=1,
                                   padding=1
                                  ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.Dropout(p=0.2)
        )
        # LSTM block
        hidden_size = 128
        self.lstm = nn.LSTM(input_size=192,hidden_size=hidden_size,bidirectional=True, batch_first=True)
        self.dropout_lstm = nn.Dropout(p=0.3)
        self.attention_linear = nn.Linear(2*hidden_size,1) # 2*hidden_size for the 2 outputs of bidir LSTM
        # Linear softmax layer
        self.out_linear = nn.Linear(2*hidden_size,1)
    def forward(self,x):
        batch_size, time_steps, height, width = x.size()
        x = x.view(batch_size * time_steps, 1, height, width)
        conv_embedding = self.conv2Dblock(x)
        conv_embedding = conv_embedding.view(batch_size, time_steps, -1, conv_embedding.size(2), conv_embedding.size(3))
        conv_embedding = torch.flatten(conv_embedding, start_dim=2) # do not flatten batch dimension and time
        lstm_embedding, (h,c) = self.lstm(conv_embedding)
        lstm_embedding = self.dropout_lstm(lstm_embedding)
        # lstm_embedding (batch, time, hidden_size*2)
        batch_size,T,_ = lstm_embedding.shape
        attention_weights = [None]*T
        for t in range(T):
            embedding = lstm_embedding[:,t,:]
            attention_weights[t] = self.attention_linear(embedding)
        attention_weights_norm = nn.functional.softmax(torch.stack(attention_weights,-1),dim=-1)
        attention = torch.bmm(attention_weights_norm,lstm_embedding) # (Bx1xT)*(B,T,hidden_size*2)=(B,1,2*hidden_size)
        attention = torch.squeeze(attention, 1)
        prediction = self.out_linear(attention)
        
        return prediction

In [ ]:
# def loss_fnc(predictions, targets):
#     return nn.MSELoss()(input=predictions,target=targets)

In [ ]:
# def make_train_step(model, loss_fnc, optimizer):
#     def train_step(X,Y):
#         # set model to train mode
#         model.train()
#         # forward pass
#         output = model(X)
#         # compute loss
#         loss = loss_fnc(output, Y)
#         # compute gradients
#         loss.backward()
#         # update parameters and zero gradients
#         optimizer.step()
#         optimizer.zero_grad()
#         return loss.item()

#     return train_step

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:

EPOCHS=500
DATASET_SIZE = X_train.shape[0]
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Selected device is {}'.format(device))
model = MyModel().to(device)
print('Number of trainable params: ',sum(p.numel() for p in model.parameters()))

optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
# train_step = make_train_step(model, loss_fnc, optimizer=OPTIMIZER)
loss_fnc = nn.MSELoss()
losses=[]
X_tensor = torch.tensor(X_train, device=device).float()
Y_tensor = torch.tensor(Y_train, device=device).float()
model.train()
for epoch in range(EPOCHS):
    # forward pass
    output = model(X_tensor)
    # compute loss
    loss = loss_fnc(output, Y_tensor)
    # compute gradients
    loss.backward()
    # update parameters and zero gradients
    optimizer.step()
    optimizer.zero_grad()
    losses.append(loss.item())
#     print('')
    print(f"Epoch {epoch} --> loss:{loss:.4f}")


In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# scaler = MinMaxScaler()
# X = scaler.fit_transform(X)
# X = X.reshape(len(videos), max_frames, all_videos.shape[1]-1)

# onehot_encoder = OneHotEncoder(sparse=False)
# y = onehot_encoder.fit_transform(y.reshape(-1, 1))
# y=to_categorical(y)
# num_classes = len(np.unique(y))


In [ ]:
# print(len(X))
# print(len(X[0]))
# print(X[0][0].shape)

In [ ]:
# # Define the data iterator function
# def data_iterator(feature_matrix, labels, batch_size=32):
#     num_samples = feature_matrix.shape[0]
#     num_batches = num_samples // batch_size

#     while True:
#         # Shuffle the data at the beginning of each epoch
#         indices = np.random.permutation(num_samples)
#         feature_matrix = feature_matrix[indices]
#         labels = labels[indices]

#         for batch_idx in range(num_batches):
#             start_idx = batch_idx * batch_size
#             end_idx = (batch_idx + 1) * batch_size
#             batch_x = feature_matrix[start_idx:end_idx]
#             batch_y = labels[start_idx:end_idx]
#             yield batch_x, batch_y


In [ ]:

# #*******************
# # Define inputs for attention and LSTM
# X=np.stack(X)
# input_data = Input(shape=(max_frames, image_size, image_size, 1))  # Input shape (time steps, features)
# conv_output = Conv3D(filters=2, kernel_size=(7, 7, 7), activation='relu')(input_data)
# conv_output = Conv3D(filters=2, kernel_size=(7, 7, 7), activation='relu')(conv_output)
# conv_output = Conv3D(filters=2, kernel_size=(5, 5, 5), activation='relu')(conv_output)
# conv_output = Conv3D(filters=2, kernel_size=(5, 5, 5), activation='relu')(conv_output)
# conv_output = Conv3D(filters=2, kernel_size=(3, 3, 3), activation='relu')(conv_output)
# conv_output = Conv3D(filters=2, kernel_size=(3, 3, 3), activation='relu')(conv_output)
# reshaped_output = Reshape((-1, 2 * 6 * 6))(conv_output)

# # Attention block
# attn_output = MultiHeadAttention(num_heads=2, key_dim=2 * 6 * 6)(reshaped_output, reshaped_output)
# attn_output = attn_output + reshaped_output  # Add skip connection

# # LSTM layer
# lstm_output = LSTM(32)(attn_output)  # Define the number of LSTM units and adjust as needed

# # Define the number of output classes
# # num_classes = y.shape[1]  # Number of columns in the one-hot encoded label matrix

# # Dense layers for classification
# output = Dense(1)(lstm_output)

# # Define the model
# model = Model(inputs=input_data, outputs=output)
# #*******************

# model.compile(loss='mean_squared_error', optimizer='adam')
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# # Train the model
# model.fit(X_train, y_train, epochs=10, batch_size=32)

# # batch_size = 32
# # epochs = 10
# # data_gen = data_iterator(X_train, y_train, batch_size)
# # steps_per_epoch = X_train.shape[0] // batch_size



# # Evaluate the model
# mse = model.evaluate(X_test, y_test)
# print("Mean Squared Error:", mse)


# for epoch in range(epochs):
#     print('Epoch:', epoch + 1)
    
#     # Train the model for one epoch
#     model.fit(data_gen, steps_per_epoch=steps_per_epoch)

# predictions = model.predict(X_test)
# predicted_labels = np.argmax(predictions, axis=1)
# # Calculate test accuracy
# accuracy = accuracy_score(np.argmax(y_test, axis=1), predicted_labels)
# print("Test Accuracy:", accuracy)